In [86]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.optimizers import Adam, SGD

In [25]:
train_df = pd.read_csv('news.csv')

In [26]:
train_df.head()

,Unnamed: 0,title,text,label
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL


## EDA

In [27]:
train_df_shuffled.head()

,Unnamed: 0,title,text,label
1357,9957,"American Dream, Revisited",Will Trump pull a Brexit times ten? What would...,FAKE
2080,7596,Clintons Are Under Multiple FBI Investigations...,Clintons Are Under Multiple FBI Investigations...,FAKE
2718,8905,The FBI Can’t Actually Investigate a Candidate...,Dispatches from Eric Zuesse This piece is cros...,FAKE
812,8752,Confirmed: Public overwhelmingly (10-to-1) say...,Print \n[Ed. – Every now and then the facade c...,FAKE
4886,7804,Nanny In Jail After Force Feeding Baby To Death,Nanny In Jail After Force Feeding Baby To Deat...,FAKE


In [28]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6335 entries, 0 to 6334
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  6335 non-null   int64 
 1   title       6335 non-null   object
 2   text        6335 non-null   object
 3   label       6335 non-null   object
dtypes: int64(1), object(3)
memory usage: 198.1+ KB


In [29]:
train_df['label'].value_counts()

REAL    3171
FAKE    3164
Name: label, dtype: int64

In [30]:
train_df['label'] = train_df['label'].replace(to_replace='FAKE', value=0)
train_df['label'] = train_df['label'].replace(to_replace='REAL', value=1)
train_df

,Unnamed: 0,title,text,label
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",0
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,0
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,1
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",0
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,1
...,...,...,...,...
6330,4490,State Department says it can't find emails fro...,The State Department told the Republican Natio...,1
6331,8062,The ‘P’ in PBS Should Stand for ‘Plutocratic’ ...,The ‘P’ in PBS Should Stand for ‘Plutocratic’ ...,0
6332,8622,Anti-Trump Protesters Are Tools of the Oligarc...,Anti-Trump Protesters Are Tools of the Oligar...,0
6333,4021,"In Ethiopia, Obama seeks progress on peace, se...","ADDIS ABABA, Ethiopia —President Obama convene...",1


In [32]:
train_df = train_df.drop('Unnamed: 0', axis=1)
train_df

,title,text,label
0,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",0
1,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,0
2,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,1
3,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",0
4,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,1
...,...,...,...
6330,State Department says it can't find emails fro...,The State Department told the Republican Natio...,1
6331,The ‘P’ in PBS Should Stand for ‘Plutocratic’ ...,The ‘P’ in PBS Should Stand for ‘Plutocratic’ ...,0
6332,Anti-Trump Protesters Are Tools of the Oligarc...,Anti-Trump Protesters Are Tools of the Oligar...,0
6333,"In Ethiopia, Obama seeks progress on peace, se...","ADDIS ABABA, Ethiopia —President Obama convene...",1


In [38]:
train_df['original_text'] = train_df['title'] + train_df['text']

In [41]:
train_df.head()

,title,text,label,original_text
0,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",0,"You Can Smell Hillary’s FearDaniel Greenfield,..."
1,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,0,Watch The Exact Moment Paul Ryan Committed Pol...
2,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,1,Kerry to go to Paris in gesture of sympathyU.S...
3,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",0,Bernie supporters on Twitter erupt in anger ag...
4,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,1,The Battle of New York: Why This Primary Matte...


In [44]:
# shuffeling train data
train_df_shuffled = train_df.sample(frac=1, random_state=42)

## **Priniting some random news**

In [49]:
import random
random_index = random.randint(0, len(train_df)-5)

for row in train_df_shuffled[['original_text', 'label']][random_index:random_index+1].itertuples():
  _, text, target = row
  print(f"Target: {target}", '(Real News)' if target > 0 else '(Fake News)')
  print(f"News: {text}")
  print('---\n')

Target: 1 (Real News)
News: Obama takes fire forJindal, the Republican governor of Louisiana, referred to Obama's comments as a "history lesson" that ignores "the issue right in front of his nose."

"We will be happy to keep an eye out for runaway Christians, but it would be nice if he would face the reality of the situation today," Jindal said in a statement out Friday. "The Medieval Christian threat is under control, Mr. President. Please deal with the Radical Islamic threat today."

Santorum, who ran for president in 2012 as well, rebuked Obama for ignoring the threat of ISIS and criticizing people of faith.

"Today's remarks by the President were inappropriate and his choice of venue was insulting to every person of faith at a time when Christians are being crucified, beheaded, and persecuted across the Middle East," the former Pennsylvania senator said in a statement.

The comments, which were intended to illustrate the potential for religion to be used for both good and bad aims,

## Train validation split

In [50]:
from sklearn.model_selection import train_test_split

In [51]:
train_news, val_news, train_label, val_label = train_test_split(train_df_shuffled['original_text'], train_df_shuffled['label'], test_size=0.2, random_state=101)

In [55]:
train_news.shape, val_news.shape

((5068,), (1267,))

In [58]:
round(sum([len(i.split()) for i in train_news])/ len(train_news))

792

## Convertin text to numbers

* Vectorization Layer
* Embedding Layer

In [56]:
from tensorflow.keras.layers import TextVectorization, Embedding

In [59]:
max_vocab_length = 10000
max_length = 792

text_vectorizer = TextVectorization(
    max_tokens=max_vocab_length,
    output_mode='int',
    output_sequence_length=max_length,
)

In [60]:
text_vectorizer.adapt(train_news)

In [68]:
embeddint = Embedding(
    input_dim = max_vocab_length, 
    output_dim = 128,
    input_length = max_length,
)

## Functions

In [77]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def calculate_result(y_true, y_pred):
  accuracy = accuracy_score(y_true, y_pred)
  model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(y_true, y_pred, average='weighted')
  model_results = {
      'accurecy': accuracy,
      'precision': model_precision,
      'recall': model_recall,
      'f1-score': model_f1,
  }

  return model_results

## Callbacks

In [80]:
save_dir = 'model_logs'
from helper_functions import create_tensorboard_callback

## Models

### 0] NaiveBase Model

In [69]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer

In [70]:
model_0 = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('clf', MultinomialNB()),
])

In [71]:
model_0.fit(train_news, train_label)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [73]:
model_0.score(val_news, val_label)

0.7868981846882399

In [74]:
model_0_perd = model_0.predict(val_news) 

In [79]:
model_0_result = calculate_result(val_label, model_0_perd)
model_0_result

{'accurecy': 0.7868981846882399,
 'precision': 0.8428623426953119,
 'recall': 0.7868981846882399,
 'f1-score': 0.7800005452730413}

## 1] Simple Dense layer model

In [84]:
inputs = layers.Input(shape=(1,), dtype=tf.string)
x = text_vectorizer(inputs)
x = embeddint(x)
x = layers.GlobalAvgPool1D()(x)
x = layers.Dense(64, activation='relu')(x)
outputs = layers.Dense(1, activation='sigmoid')(x)

model_1 = tf.keras.Model(inputs, outputs)

In [85]:
model_1.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 1)]               0         
_________________________________________________________________
text_vectorization (TextVect (None, 792)               0         
_________________________________________________________________
embedding (Embedding)        (None, 792, 128)          1280000   
_________________________________________________________________
global_average_pooling1d_1 ( (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 65        
Total params: 1,288,321
Trainable params: 1,288,321
Non-trainable params: 0
_________________________________________________

In [87]:
model_1.compile(
    loss = 'binary_crossentropy',
    optimizer = Adam(),
    metrics = ['accuracy'],
)

history_1 = model_1.fit(
    train_news,
    train_label,
    epochs = 10,
    validation_data = (val_news, val_label),
    callbacks = [
        create_tensorboard_callback(save_dir, 'model_1')
    ]
)

Saving TensorBoard log files to: model_logs/model_1/20211010-210653
Epoch 1/10
159/159 [==============================] - 8s 46ms/step - loss: 0.6002 - accuracy: 0.7188 - val_loss: 0.3963 - val_accuracy: 0.8863
Epoch 2/10
159/159 [==============================] - 7s 46ms/step - loss: 0.2981 - accuracy: 0.8905 - val_loss: 0.2672 - val_accuracy: 0.9140
Epoch 3/10
159/159 [==============================] - 7s 46ms/step - loss: 0.1914 - accuracy: 0.9307 - val_loss: 0.2830 - val_accuracy: 0.8698
Epoch 4/10
159/159 [==============================] - 7s 45ms/step - loss: 0.1354 - accuracy: 0.9544 - val_loss: 0.2518 - val_accuracy: 0.8919
Epoch 5/10
159/159 [==============================] - 7s 43ms/step - loss: 0.0954 - accuracy: 0.9706 - val_loss: 0.2123 - val_accuracy: 0.9369
Epoch 6/10
159/159 [==============================] - 7s 45ms/step - loss: 0.0724 - accuracy: 0.9803 - val_loss: 0.2169 - val_accuracy: 0.9353
Epoch 7/10
159/159 [==============================] - 7s 44ms/step - loss:

In [91]:
model_1_pred = model_1.predict(val_news)
model_1_pred = tf.squeeze(tf.round(model_1_pred))
model_1_result = calculate_result(val_label, model_1_pred)
model_1_result

{'accurecy': 0.9163378058405682,
 'precision': 0.9192014215446093,
 'recall': 0.9163378058405682,
 'f1-score': 0.9163404116718634}

## 2] LSTM Model

In [92]:
inputs = layers.Input(shape=(1,), dtype=tf.string)
x = text_vectorizer(inputs)
x = embeddint(x)
x = layers.LSTM(64, return_sequences=True)(x)
x = layers.LSTM(64)(x)
x = layers.Dense(64, activation='relu')(x)
outputs = layers.Dense(1, activation='sigmoid')(x)

model_2 = tf.keras.Model(inputs, outputs)

In [93]:
model_2.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 1)]               0         
_________________________________________________________________
text_vectorization (TextVect (None, 792)               0         
_________________________________________________________________
embedding (Embedding)        (None, 792, 128)          1280000   
_________________________________________________________________
lstm (LSTM)                  (None, 792, 64)           49408     
_________________________________________________________________
lstm_1 (LSTM)                (None, 64)                33024     
_________________________________________________________________
dense_3 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 65  

In [ ]:
model_2.compile(
    loss = 'binary_crossentropy',
    optimizer = Adam(),
    metrics = ['accuracy'],
)

history_2 = model_2.fit(
    train_news,
    train_label,
    epochs = 10,
    validation_data = (val_news, val_label),
    callbacks= [
        create_tensorboard_callback(save_dir, 'model_2')
    ]
)

Saving TensorBoard log files to: model_logs/model_2/20211010-211527
Epoch 1/10
159/159 [==============================] - 156s 960ms/step - loss: 0.6505 - accuracy: 0.6044 - val_loss: 0.4191 - val_accuracy: 0.8729
Epoch 2/10
159/159 [==============================] - 190s 1s/step - loss: 0.3675 - accuracy: 0.8698 - val_loss: 0.4260 - val_accuracy: 0.8406
Epoch 3/10
142/159 [=========================>....] - ETA: 16s - loss: 0.3270 - accuracy: 0.8836

ERROR:asyncio:Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "C:\Users\vishal\anaconda3\lib\asyncio\events.py", line 81, in _run
    self._context.run(self._callback, *self._args)
  File "C:\Users\vishal\anaconda3\lib\asyncio\selector_events.py", line 120, in _read_from_self
    data = self._ssock.recv(4096)
ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host


143/159 [=========================>....] - ETA: 15s - loss: 0.3275 - accuracy: 0.8837

ERROR:asyncio:Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "C:\Users\vishal\anaconda3\lib\asyncio\events.py", line 81, in _run
    self._context.run(self._callback, *self._args)
  File "C:\Users\vishal\anaconda3\lib\asyncio\selector_events.py", line 120, in _read_from_self
    data = self._ssock.recv(4096)
ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host


144/159 [==========================>...] - ETA: 15s - loss: 0.3279 - accuracy: 0.8839

...

 44/159 [=======>......................] - ETA: 19:11 - loss: 0.2696 - accuracy: 0.9112

ERROR:asyncio:Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "C:\Users\vishal\anaconda3\lib\asyncio\events.py", line 81, in _run
    self._context.run(self._callback, *self._args)
  File "C:\Users\vishal\anaconda3\lib\asyncio\selector_events.py", line 120, in _read_from_self
    data = self._ssock.recv(4096)
ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host


 45/159 [=======>......................] - ETA: 19:01 - loss: 0.2715 - accuracy: 0.9111

ERROR:asyncio:Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "C:\Users\vishal\anaconda3\lib\asyncio\events.py", line 81, in _run
    self._context.run(self._callback, *self._args)
  File "C:\Users\vishal\anaconda3\lib\asyncio\selector_events.py", line 120, in _read_from_self
    data = self._ssock.recv(4096)
ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host


 46/159 [=======>......................] - ETA: 18:51 - loss: 0.2735 - accuracy: 0.9103